# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [1]:
activity=4310029237#4088494919
date='November 8, 2020'
LAST=True

In [2]:
import helium
import pandas as pd
import os
import time
import sys
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
bkp=pd.read_json(db_file)
if not LAST and not bkp[bkp.Info.astype(str).str.contains(date)].empty:
    sys.exit('Check if already exists in bkp')

In [4]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [5]:
ll[0]

,Unnamed: 0,Avg,Max
0,Speed,13.2km/h,72.0km/h
1,Calories,3951,3951
2,Elapsed Time,6:22:12,6:22:12


In [6]:
sgmt

,Name,Time,Speed,Power,Info
0,Jumbo-Sena 8.09km 56m -0.7%,17:53,27.2km/h,255W,"{'Calories': {'Avg': '3951', 'Max': '3951'}, '..."
1,Carrefour Vegas- Puente de la 30 4.89km 37m ...,8:39,33.9km/h,319W,"{'Calories': {'Avg': '3951', 'Max': '3951'}, '..."
2,Retorno Ayura -Pte 12 sur 0.84km 7m -0.8%,1:27,35.1km/h,310W,"{'Calories': {'Avg': '3951', 'Max': '3951'}, '..."
3,12 sur (Aguacatala) - La 4 sur 0.98km 13m -...,1:43,34.5km/h,302W,"{'Calories': {'Avg': '3951', 'Max': '3951'}, '..."
4,12 sur (Aguacatala) - La 10 2.09km 19m -0.7%,3:42,34.0km/h,312W,"{'Calories': {'Avg': '3951', 'Max': '3951'}, '..."
5,4 sur- La 30 3.07km 24m -0.7%,5:23,34.2km/h,328W,"{'Calories': {'Avg': '3951', 'Max': '3951'}, '..."
6,4 sur- La 10 1.10km 11m -0.5%,1:57,33.9km/h,322W,"{'Calories': {'Avg': '3951', 'Max': '3951'}, '..."
7,monterey-bancolombia 0.74km 9m -0.9%,1:21,33.2km/h,307W,"{'Calories': {'Avg': '3951', 'Max': '3951'}, '..."
8,La 10 hasta 33 2.98km 26m -0.5%,8:09,22.0km/h,207W,"{'Calories': {'Avg': '3951', 'Max': '3951'}, '..."
9,Sprint Monterrey - Bancolombia 1.41km 16m -...,2:28,34.4km/h,336W,"{'Calories': {'Avg': '3951', 'Max': '3951'}, '..."


In [7]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [8]:
df.to_json(db_file)

### Save notebook here

In [9]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [10]:
!bash commit.sh

[master 651c78a] Morning Activity 6:43 AM on Sunday, November 8, 2020
 2 files changed, 250 insertions(+), 925 deletions(-)
 rewrite Strava.ipynb (85%)
 rewrite db/strava.json (69%)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 24.51 KiB | 1003.00 KiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:restrepo/strava.git
   d53b8bd..651c78a  master -> master


In [11]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()

Close browser? 
